# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
from scipy.stats import percentileofscore as score
import xlsxwriter
from datetime import date, timedelta
import json


In [3]:
api_key= "UTyHVduohyBpg6IFqApjemQi7MtN1dTB"

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv("SP500.csv")
#stocks = pd.read_csv("sp_500_stocks.csv")


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [7]:
symbol = "SP"
today = date.today()
one_year_ago = today - timedelta(days=(365))
api_url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?from={one_year_ago}&to={today}&apikey={api_key}"
data = requests.get(api_url).json()
first_day = data["historical"][-1]
recent_day = data["historical"][0]
start_price = first_day.get('close')
recent_price = recent_day.get('close')
yearly_per = (recent_price / start_price)
data

KeyError: 'historical'

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:

# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 5))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
symbol_strings

MMM,AOS,ABT,ABBV,ACN
ATVI,AYI,ADBE,AAP,AMD
AES,AET,AMG,AFL,A
APD,AKAM,ALK,ALB,ARE
ALXN,ALGN,ALLE,AGN,ADS
LNT,ALL,GOOGL,GOOG,MO
AMZN,AEE,AAL,AEP,AXP
AIG,AMT,AWK,AMP,ABC
AME,AMGN,APH,APC,ADI
ANDV,ANSS,ANTM,AON,APA
AIV,AAPL,AMAT,APTV,ADM
ARNC,AJG,AIZ,T,ADSK
ADP,AZO,AVB,AVY,BHGE
BLL,BAC,BAX,BBT,BDX
BRK.B,BBY,BIIB,BLK,HRB
BA,BWA,BXP,BSX,BHF
BMY,AVGO,BF.B,CHRW,CA
COG,CDNS,CPB,COF,CAH
KMX,CCL,CAT,CBOE,CBG
CBS,CELG,CNC,CNP,CTL
CERN,CF,SCHW,CHTR,CHK
CVX,CMG,CB,CHD,CI
XEC,CINF,CTAS,CSCO,C
CFG,CTXS,CME,CMS,KO
CTSH,CL,CMCSA,CMA,CAG
CXO,COP,ED,STZ,GLW
COST,COTY,CCI,CSRA,CSX
CMI,CVS,DHI,DHR,DRI
DVA,DE,DAL,XRAY,DVN
DLR,DFS,DISCA,DISCK,DISH
DG,DLTR,D,DOV,DWDP
DPS,DTE,DUK,DRE,DXC
ETFC,EMN,ETN,EBAY,ECL
EIX,EW,EA,EMR,ETR
EVHC,EOG,EQT,EFX,EQIX
EQR,ESS,EL,RE,ES
EXC,EXPE,EXPD,ESRX,EXR
XOM,FFIV,FB,FAST,FRT
FDX,FIS,FITB,FE,FISV
FLIR,FLS,FLR,FMC,FL
F,FTV,FBHS,BEN,FCX
GPS,GRMN,IT,GD,GE
GGP,GIS,GM,GPC,GILD
GPN,GS,GT,GWW,HAL
HBI,HOG,HRS,HIG,HAS
HCA,HCP,HP,HSIC,HES
HPE,HLT,HOLX,HD,HON
HRL,HST,HPQ,HUM,HBAN
HII,IDXX

['MMM,AOS,ABT,ABBV,ACN',
 'ATVI,AYI,ADBE,AAP,AMD',
 'AES,AET,AMG,AFL,A',
 'APD,AKAM,ALK,ALB,ARE',
 'ALXN,ALGN,ALLE,AGN,ADS',
 'LNT,ALL,GOOGL,GOOG,MO',
 'AMZN,AEE,AAL,AEP,AXP',
 'AIG,AMT,AWK,AMP,ABC',
 'AME,AMGN,APH,APC,ADI',
 'ANDV,ANSS,ANTM,AON,APA',
 'AIV,AAPL,AMAT,APTV,ADM',
 'ARNC,AJG,AIZ,T,ADSK',
 'ADP,AZO,AVB,AVY,BHGE',
 'BLL,BAC,BAX,BBT,BDX',
 'BRK.B,BBY,BIIB,BLK,HRB',
 'BA,BWA,BXP,BSX,BHF',
 'BMY,AVGO,BF.B,CHRW,CA',
 'COG,CDNS,CPB,COF,CAH',
 'KMX,CCL,CAT,CBOE,CBG',
 'CBS,CELG,CNC,CNP,CTL',
 'CERN,CF,SCHW,CHTR,CHK',
 'CVX,CMG,CB,CHD,CI',
 'XEC,CINF,CTAS,CSCO,C',
 'CFG,CTXS,CME,CMS,KO',
 'CTSH,CL,CMCSA,CMA,CAG',
 'CXO,COP,ED,STZ,GLW',
 'COST,COTY,CCI,CSRA,CSX',
 'CMI,CVS,DHI,DHR,DRI',
 'DVA,DE,DAL,XRAY,DVN',
 'DLR,DFS,DISCA,DISCK,DISH',
 'DG,DLTR,D,DOV,DWDP',
 'DPS,DTE,DUK,DRE,DXC',
 'ETFC,EMN,ETN,EBAY,ECL',
 'EIX,EW,EA,EMR,ETR',
 'EVHC,EOG,EQT,EFX,EQIX',
 'EQR,ESS,EL,RE,ES',
 'EXC,EXPE,EXPD,ESRX,EXR',
 'XOM,FFIV,FB,FAST,FRT',
 'FDX,FIS,FITB,FE,FISV',
 'FLIR,FLS,FLR,FMC,FL',
 'F,

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [43]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
rows_to_add = [] # Use a list to collect data rows efficiently

# Loop through each batch of tickers
for symbol_string in symbol_strings:
    # 1. Corrected the URL construction and API call variable
    batch_url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol_string}?from={one_year_ago}&to={today}&apikey={api_key}"
    response = requests.get(batch_url)

    if response.status_code == 200:
        data = response.json()
        
        # 2. Correctly parse the 'historicalStockList' from the JSON response
        if 'historicalStockList' in data:
            for stock_data in data['historicalStockList']:
                symbol = stock_data.get('symbol')
                historical_data = stock_data.get('historical', [])

                # Ensure there is enough data to calculate a return
                if len(historical_data) > 1:
                    # FMP returns data newest-first
                    
                    recent_price = historical_data[0]['close']
                    start_price = historical_data[250]['close']
                    
                    # 3. Calculated percentage return, not absolute change
                    price_return = (recent_price - start_price) / start_price
                    
                    rows_to_add.append({
                        'Ticker': symbol,
                        'Price': recent_price,
                        'One-Year Price Return': price_return,
                        'Number of Shares to Buy': 'N/A'
                    })
# 4. Create the DataFrame from the list of rows in a single, efficient step
final_df = pd.DataFrame(rows_to_add, columns=my_columns)

print(final_df)

IndexError: list index out of range

In [28]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
rows_to_add = [] # Use a list to collect data rows efficiently

# Loop through each batch of tickers
for symbol_string in symbol_strings:
    # 1. Corrected the URL construction and API call variable
    batch_url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol_string}?from={one_year_ago}&to={today}&apikey={api_key}"
    response = requests.get(batch_url)

    if response.status_code == 200:
        data = response.json()
        
        # 2. Correctly parse the 'historicalStockList' from the JSON response
        if 'historicalStockList' in data:
            for stock_data in data:
                symbol = stock_data["symbol"]
                historical_data = stock_data.get('historical', [])
                #print(historical_data)
                #break
                # Ensure there is enough data to calculate a return
                if len(historical_data) > 1:
                    # FMP returns data newest-first
                    
                    recent_price = historical_data[0]['close']
                    start_price = historical_data[250]['close']
                    
                    # 3. Calculated percentage return, not absolute change
                    price_return = (recent_price - start_price) / start_price
                    
                    rows_to_add.append({
                        'Ticker': symbol,
                        'Price': recent_price,
                        'One-Year Price Return': price_return,
                        'Number of Shares to Buy': 'N/A'
                    })
                    
# 4. Create the DataFrame from the list of rows in a single, efficient step
#final_df = pd.DataFrame(rows_to_add, columns=my_columns)

#print(final_df)

TypeError: string indices must be integers, not 'str'

In [44]:
1

1

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_df.sort_values("One-Year Price Return", ascending = False, inplace = True)
final_df = final_df[:50]
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
11,GL,140.3000,0.523841,N/A
12,GLW,62.2900,0.459466,N/A
5,CME,277.1626,0.393337,N/A
24,RTX,157.2100,0.373733,N/A
23,RSG,245.8100,0.279194,N/A
7,CMI,365.4000,0.254808,N/A
20,ROL,57.5350,0.209227,N/A
13,GM,52.8500,0.197055,N/A
15,MCK,713.6200,0.176213,N/A
18,MDT,92.4600,0.155750,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def budget():
    portfolio_size = input("What is your portfolio size? ")
    
    while True:
        try:
            val = float(portfolio_size)
            print(portfolio_size)
            break  # Exit the loop if conversion succeeds
        except ValueError:
            print("Please enter a valid number.")
            portfolio_size = input("What is your portfolio size? ")    
    return val

In [9]:
position_size = budget()/len(final_df.index)
for i in range(len(final_df.index)):    
    final_df["Number of Shares to Buy"] = (position_size / final_df["Price"]).apply(math.floor)
final_df

What is your portfolio size?  321897932817


321897932817


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
11,GL,140.3000,0.523841,76478482
12,GLW,62.2900,0.459466,172257683
5,CME,277.1626,0.393337,38713488
24,RTX,157.2100,0.373733,68252217
23,RSG,245.8100,0.279194,43651320
7,CMI,365.4000,0.254808,29364890
20,ROL,57.5350,0.209227,186493979
13,GM,52.8500,0.197055,203026132
15,MCK,713.6200,0.176213,15035917
18,MDT,92.4600,0.155750,116049438


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    '1-Year Return',
    '1-Year Percentile',
    '6-Month Return',
    '6-Month Percentile',
    '3-Month Return',
    '3-Month Percentile',
    '1-Month Return',
    '1-Month Percentile',
    'HQM Score'
]

In [11]:


rows_to_add = []

# 1. Use a dictionary to define periods and their day counts for a clean loop
time_periods = {
    '1-Year': 250,
    '6-Month': 124,
    '3-Month': 61,
    '1-Month': 19
}

for symbol_string in symbol_strings:
    batch_url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol_string}?from={one_year_ago}&to={today}&apikey={api_key}"
    response = requests.get(batch_url)

    if response.status_code == 200:
        data = response.json()
        if 'historicalStockList' in data:
            for stock_data in data['historicalStockList']:
                symbol = stock_data.get('symbol')
                historical_data = stock_data.get('historical', [])

                if not historical_data:
                    continue

                recent_price = historical_data[0]['close']
                returns_data = {} # Dictionary to hold results for THIS stock

                # 2. Loop through each time period instead of repeating code
                for period_name, days_back in time_periods.items():
                    # 3. Check length *before* trying to access the index to prevent crashes
                    if len(historical_data) > days_back:
                        prev_price = historical_data[days_back]['close']
                        return_percent = (recent_price - prev_price) / prev_price
                        returns_data[f'{period_name} Return'] = return_percent
                        returns_data[f'{period_name} Percentile'] = "N/A"
                    else:
                        returns_data[f'{period_name} Return'] = None # Mark as None if not enough data
                        returns_data[f'{period_name} Percentile'] = "N/A"

                # Append a single, complete dictionary for the row
                rows_to_add.append({
                    "Ticker": symbol,
                    "Price": recent_price,
                    "Number of Shares to Buy": "N/A",
                    "HQM Score": "N/A",
                    **returns_data  # Safely unpacks all calculated returns
                })

# Create the DataFrame
hqm_df = pd.DataFrame(rows_to_add, columns=hqm_columns)
hqm_df

,Ticker,Price,Number of Shares to Buy,1-Year Return,1-Year Percentile,6-Month Return,6-Month Percentile,3-Month Return,3-Month Percentile,1-Month Return,1-Month Percentile,HQM Score
0,MMM,144.41,N/A,None,N/A,-0.037523,N/A,0.025639,N/A,-0.048933,N/A,N/A
1,AOS,70.41,N/A,None,N/A,0.050268,N/A,0.030290,N/A,0.034528,N/A,N/A
2,ABT,127.29,N/A,None,N/A,-0.009031,N/A,-0.040045,N/A,-0.047943,N/A,N/A
3,ABBV,195.22,N/A,None,N/A,0.026717,N/A,-0.004335,N/A,0.040452,N/A,N/A
4,ACN,255.37,N/A,None,N/A,-0.337063,N/A,-0.170095,N/A,-0.150296,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
427,XYL,142.18,N/A,None,N/A,0.157818,N/A,0.162266,N/A,0.085344,N/A,N/A
428,YUM,145.36,N/A,None,N/A,0.105399,N/A,-0.008323,N/A,-0.033254,N/A,N/A
429,ZBH,90.62,N/A,None,N/A,-0.162167,N/A,0.001547,N/A,-0.021065,N/A,N/A
430,ZION,52.08,N/A,None,N/A,-0.074133,N/A,0.120000,N/A,-0.055838,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [12]:
time_periods = [
    "1-Year",
    "6-Month",
    "3-Month",
    "1-Month",
]

for time_period in time_periods:
    # Convert to numeric, coercing errors to NaN
    hqm_df[f"{time_period} Return"] = pd.to_numeric(hqm_df[f"{time_period} Return"], errors='coerce')
    
    # Drop NaN values for the percentile calculation
    valid_returns = hqm_df[f"{time_period} Return"].dropna()
    
    if len(valid_returns) > 0:  # Only calculate if we have valid data
        # Calculate percentiles using numpy's percentileofscore
        hqm_df[f"{time_period} Percentile"] = hqm_df[f"{time_period} Return"].apply(
            lambda x: score(valid_returns, x) if pd.notnull(x) else np.nan
        )
    else:
        hqm_df[f"{time_period} Percentile"] = np.nan
hqm_df

,Ticker,Price,Number of Shares to Buy,1-Year Return,1-Year Percentile,6-Month Return,6-Month Percentile,3-Month Return,3-Month Percentile,1-Month Return,1-Month Percentile,HQM Score
0,MMM,144.41,N/A,NaN,NaN,-0.037523,44.028103,0.025639,45.920746,-0.048933,32.250580,N/A
1,AOS,70.41,N/A,NaN,NaN,0.050268,63.466042,0.030290,47.086247,0.034528,76.102088,N/A
2,ABT,127.29,N/A,NaN,NaN,-0.009031,51.288056,-0.040045,23.543124,-0.047943,33.642691,N/A
3,ABBV,195.22,N/A,NaN,NaN,0.026717,58.782201,-0.004335,34.265734,0.040452,78.654292,N/A
4,ACN,255.37,N/A,NaN,NaN,-0.337063,3.747073,-0.170095,4.662005,-0.150296,5.800464,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
427,XYL,142.18,N/A,NaN,NaN,0.157818,83.840749,0.162266,80.419580,0.085344,92.343387,N/A
428,YUM,145.36,N/A,NaN,NaN,0.105399,76.112412,-0.008323,33.100233,-0.033254,40.835267,N/A
429,ZBH,90.62,N/A,NaN,NaN,-0.162167,17.798595,0.001547,37.529138,-0.021065,47.563805,N/A
430,ZION,52.08,N/A,NaN,NaN,-0.074133,33.723653,0.120000,71.794872,-0.055838,28.770302,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [13]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row,f"{time_period} Percentile"])
    hqm_df.loc[row,"HQM Score"] = mean(momentum_percentiles)
hqm_df

,Ticker,Price,Number of Shares to Buy,1-Year Return,1-Year Percentile,6-Month Return,6-Month Percentile,3-Month Return,3-Month Percentile,1-Month Return,1-Month Percentile,HQM Score
0,MMM,144.41,N/A,NaN,NaN,-0.037523,44.028103,0.025639,45.920746,-0.048933,32.250580,NaN
1,AOS,70.41,N/A,NaN,NaN,0.050268,63.466042,0.030290,47.086247,0.034528,76.102088,NaN
2,ABT,127.29,N/A,NaN,NaN,-0.009031,51.288056,-0.040045,23.543124,-0.047943,33.642691,NaN
3,ABBV,195.22,N/A,NaN,NaN,0.026717,58.782201,-0.004335,34.265734,0.040452,78.654292,NaN
4,ACN,255.37,N/A,NaN,NaN,-0.337063,3.747073,-0.170095,4.662005,-0.150296,5.800464,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
427,XYL,142.18,N/A,NaN,NaN,0.157818,83.840749,0.162266,80.419580,0.085344,92.343387,NaN
428,YUM,145.36,N/A,NaN,NaN,0.105399,76.112412,-0.008323,33.100233,-0.033254,40.835267,NaN
429,ZBH,90.62,N/A,NaN,NaN,-0.162167,17.798595,0.001547,37.529138,-0.021065,47.563805,NaN
430,ZION,52.08,N/A,NaN,NaN,-0.074133,33.723653,0.120000,71.794872,-0.055838,28.770302,NaN


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [14]:
hqm_df.sort_values("HQM Score", ascending = False, inplace = True)
hqm_dfs = hqm_df[:50]
hqm_dfs.reset_index(drop = True, inplace = True)
hqm_dfs

,Ticker,Price,Number of Shares to Buy,1-Year Return,1-Year Percentile,6-Month Return,6-Month Percentile,3-Month Return,3-Month Percentile,1-Month Return,1-Month Percentile,HQM Score
0,MMM,144.41,N/A,NaN,NaN,-0.037523,44.028103,0.025639,45.920746,-0.048933,32.250580,NaN
1,AOS,70.41,N/A,NaN,NaN,0.050268,63.466042,0.030290,47.086247,0.034528,76.102088,NaN
2,ABT,127.29,N/A,NaN,NaN,-0.009031,51.288056,-0.040045,23.543124,-0.047943,33.642691,NaN
3,ABBV,195.22,N/A,NaN,NaN,0.026717,58.782201,-0.004335,34.265734,0.040452,78.654292,NaN
4,ACN,255.37,N/A,NaN,NaN,-0.337063,3.747073,-0.170095,4.662005,-0.150296,5.800464,NaN
5,AYI,304.83,N/A,NaN,NaN,-0.036233,44.262295,0.207152,86.480186,-0.006615,56.148492,NaN
6,ADBE,347.80,N/A,NaN,NaN,-0.207022,11.475410,-0.087283,12.121212,-0.077282,17.169374,NaN
7,AAP,52.71,N/A,NaN,NaN,0.093568,72.833724,0.657026,99.300699,-0.027850,43.155452,NaN
8,AMD,171.70,N/A,NaN,NaN,0.502582,98.829040,0.706929,99.766900,0.273739,100.000000,NaN
9,AES,13.23,N/A,NaN,NaN,0.235294,91.569087,0.289474,92.307692,0.148438,97.215777,NaN


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [75]:
position_size = budget()/len(hqm_dfs.index)
for i in range(len(hqm_dfs.index)):    
    hqm_dfs["Number of Shares to Buy"] = (position_size / hqm_dfs["Price"]).apply(math.floor)
hqm_dfs

What is your portfolio size?  59000


59000


C:\Users\Abraham\AppData\Local\Temp\ipykernel_25904\2211160842.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dfs["Number of Shares to Buy"] = (position_size / hqm_dfs["Price"]).apply(math.floor)


,Ticker,Price,Number of Shares to Buy,1-Year Return,1-Year Percentile,6-Month Return,6-Month Percentile,3-Month Return,3-Month Percentile,1-Month Return,1-Month Percentile,HQM Score
404,TPR,107.5300,10,1.664272,100.000000,0.445490,97.821351,0.522010,97.173913,0.236830,99.350649,98.586478
324,ORCL,249.4661,4,0.803674,98.017621,0.539724,99.346405,0.772784,99.565217,0.139323,96.103896,98.258285
315,NVDA,176.4950,6,0.581638,96.255507,0.426799,97.385621,0.620409,98.913043,0.151305,97.186147,97.43508
45,AVGO,298.0450,3,0.984057,99.118943,0.444366,97.603486,0.548527,97.826087,0.125803,93.939394,97.121977
352,PWR,412.3600,2,0.609398,96.916300,0.384688,96.949891,0.408863,95.217391,0.107631,91.341991,95.106393
28,AMD,180.0300,6,0.288229,79.735683,0.534129,98.910675,0.849307,100.000000,0.322680,99.783550,94.607477
179,GLW,62.6100,18,0.466963,92.290749,0.257229,92.810458,0.410771,95.434783,0.190531,97.835498,94.592872
439,WDC,70.7750,16,0.409861,88.766520,0.491570,98.257081,0.613657,98.478261,0.108631,91.774892,94.319188
40,APH,105.8250,11,0.669427,97.136564,0.525295,98.692810,0.375244,93.043478,0.086611,87.445887,94.079685
397,TEL,208.7650,5,0.340213,84.361233,0.413344,97.167756,0.426185,95.869565,0.226082,98.917749,94.079076


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [12]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

NameError: name 'writer' is not defined

## Saving Our Excel Output

As before, saving our Excel output is very easy: